# 부정 리뷰를 이용하여 부정 키워드 추출
- 긍정리뷰를 이용한 방식과 동일하게 부정리뷰에 대한 주제를 분석할것이다.
- 이후 호텔(=문서)별로 많이 분포하는 주제를 지정해주고 그 주제에 맞는 키워드를 연결할 것이다.

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data=pd.read_excel('X_train.xlsx',encoding='cp949')

In [4]:
test=pd.read_excel('X_test.xlsx',encoding='cp949')

In [5]:
test=test.iloc[np.random.randint(0,test.shape[0], 1300), :]

In [6]:
#test.to_excel('korea.xlsx', encoding = 'cp949')

# 데이터셋 구축

In [7]:
data=data.query("평점==0")

In [8]:
test=test.query("평점 == 0")
test=test.reset_index(drop=True)

In [9]:
data=data.reset_index(drop=True)

In [10]:
data['호텔이름'].nunique()

283

In [11]:
data_dict=dict()
for i in range(len(data)):
    if data['호텔이름'][i] not in data_dict.keys():
        data_dict[data['호텔이름'][i]]=data['리뷰'][i]
    else:
        data_dict[data['호텔이름'][i]]=data_dict[data['호텔이름'][i]]+' '+ data['리뷰'][i]

In [12]:
test_dict=dict()
for i in range(len(test)):
    if test['호텔이름'][i] not in test_dict.keys():
        test_dict[test['호텔이름'][i]]=test['리뷰'][i]
    else:
        test_dict[test['호텔이름'][i]]=test_dict[test['호텔이름'][i]]+' '+ test['리뷰'][i]

In [13]:
a=pd.DataFrame.from_dict([data_dict])
final_data=pd.melt(a,var_name='호텔이름', value_name='리뷰')

In [14]:
b=pd.DataFrame.from_dict([test_dict])
final_test=pd.melt(b,var_name='호텔이름', value_name='리뷰')

In [15]:
len(final_data['호텔이름'].unique())

283

In [16]:
final_test

,호텔이름,리뷰
0,신라스테이 해운대,아직 갈길이 멀다 카운터 첫인상 별로 뭔가 어수선함 서비스 받는 느낌은 포기해야함 ...
1,롯데시티호텔 제주,"부모님을 모시고 간 제주도 여행에서 숙박시설은 중요한 부분이였습니다 다만, 일정을 ..."
2,왕의지밀,먼지 냄새 가득한 한옥호텔 공사 먼지가 아직 남아 있는 텁텁한 공기가 인상적이었습니...
3,데이즈호텔 제주 서귀포,화장실에서 나는 담배 냄새와 알 수 없는 퀴퀴한 냄새가 계속 올라옴 카펫이 매우 더...
4,세인트존스 호텔,"오션뷰 날씨가 너무 좋았고, 침대도 넓고 편했고, 조용했습니다 아침에 들려오는 파도..."
...,...,...
112,르 메르디앙 서울,"서비스가 너무 좋고, 럭셔리함이 물씬 묻어나는 호텔입니다 위치적으로도 앞에 편의점도..."
113,오라카이 송도파크 호텔,"깨끗해서 편하게 잘 쉬었어요 침대 메트리스가 너무 출렁거려서 불편했고, 방 바닥 카..."
114,L7 홍대 바이 롯데,그냥 뭐 홍대에 생겼다길래 궁금해서 와봤는데 그냥 저냥 한 비즈니스 모텔 수준이고 ...
115,쉐라톤 서울 팔래스 강남 호텔,편안 하고 좋았음


# konlpy 로 TDM 만들기

In [17]:
from konlpy.tag import Komoran

In [18]:
tagger = Komoran()

In [19]:
def extract_nouns(text):
    nouns = tagger.nouns(text)
    values = [noun for noun in nouns if (len(noun)> 1 and noun!='호텔') or noun == '돈' ]
    return values

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=500, tokenizer= extract_nouns)
tdm = cv.fit_transform(final_data['리뷰'])

In [21]:
tdm_test = cv.transform(final_test['리뷰'])

In [22]:
from gensim.matutils import Sparse2Corpus

In [23]:
corpus = Sparse2Corpus(tdm.T) 
id2word = dict(enumerate(cv.get_feature_names()))

In [24]:
corpus_test = Sparse2Corpus(tdm_test.T)

In [25]:
from gensim.models.ldamodel import LdaModel

In [26]:
for num_topics in [3, 4, 5 , 6]:
    globals()['models_{}'.format(num_topics)] = LdaModel(
        corpus=corpus,
        num_topics=num_topics,
        passes=3,
        iterations=200,
        id2word=id2word,
        random_state=123)

In [28]:
from gensim.models import CoherenceModel
for i in [3, 4 , 5 , 6]:
    log_pp = globals()['models_{}'.format(i)].log_perplexity(corpus_test)
    coh = CoherenceModel(model=globals()['models_{}'.format(i)], corpus=corpus_test, coherence='u_mass')
    print(i, globals()['models_{}'.format(
        i)],"혼란도: ",2 ** (-log_pp),"응집도: ",coh.get_coherence())

3 LdaModel(num_terms=500, num_topics=3, decay=0.5, chunksize=2000) 혼란도:  150.8995872044958 응집도:  -2.3700865075146336
4 LdaModel(num_terms=500, num_topics=4, decay=0.5, chunksize=2000) 혼란도:  212.2224298376223 응집도:  -2.224970678190461
5 LdaModel(num_terms=500, num_topics=5, decay=0.5, chunksize=2000) 혼란도:  304.2731704711428 응집도:  -2.491353151137935
6 LdaModel(num_terms=500, num_topics=6, decay=0.5, chunksize=2000) 혼란도:  435.19271782446697 응집도:  -3.0073753605536298


In [30]:
for words in models_3.show_topics(formatted=False,num_words=15, num_topics = 3):
    print(words[0])
    print("******************************")
    for word_prob in words[1]:
        print(word_prob)
    print("")
    print("******************************")

0
******************************
('객실', 0.021876246)
('직원', 0.021444593)
('조식', 0.020877738)
('이용', 0.019716593)
('불편', 0.017722312)
('서비스', 0.016187826)
('친절', 0.015432852)
('만족', 0.014876312)
('시설', 0.014074148)
('수영장', 0.012849676)
('침대', 0.012305811)
('체크인', 0.01205612)
('시간', 0.011515108)
('예약', 0.010459256)
('주변', 0.01042978)

******************************
1
******************************
('이용', 0.026590155)
('조식', 0.022603415)
('친절', 0.021308422)
('객실', 0.020184787)
('직원', 0.019213274)
('위치', 0.019113775)
('만족', 0.018431483)
('시설', 0.017480072)
('불편', 0.015935313)
('주변', 0.014006235)
('청결', 0.012822192)
('가격', 0.012332948)
('서비스', 0.011404481)
('편리', 0.011287573)
('편안', 0.010805037)

******************************
2
******************************
('시설', 0.020479478)
('객실', 0.020378685)
('불편', 0.019337913)
('침대', 0.017754141)
('직원', 0.016405327)
('만족', 0.016257562)
('친절', 0.016064364)
('화장실', 0.014955933)
('주차', 0.014715003)
('가격', 0.014282473)
('냄새', 0.01391229)
('이용', 0.013577

In [31]:
topic=[]
for i in range(len(corpus)):
    doc_topic_dist = models_3.get_document_topics(corpus[i], minimum_probability=0)
    sorted_doc_topic = sorted(doc_topic_dist, key=lambda x:x[1], reverse=True)
    topic.append(sorted_doc_topic[0][0])

In [32]:
final_data2=pd.read_excel('final_data.xlsx',encoding='cp949')

In [33]:
final_data2

,호텔이름,리뷰,긍정
0,그랜드 머큐어 앰배서더 호텔 앤드 레지던스 서울 용산,만족합니다 서비스 청결 등등 모두 좋았습니다 단지 아쉬운점은 레지던스 호텔이다보니 ...,1
1,롯데 호텔 서울,로비에서부터 체크아웃 까지 정말 친절한 호텔 특별한 날에 이그제큐티브 룸 이용할 만...,1
2,남대문 호텔앤스위트,숙소가 상당히 깔끔하고 깨끗해서 좋았습니다 방 시설 좋았고 근처 인프라 좋아요 어매...,1
3,서울신라호텔,생일이라고 축하도 엄청해주시고 초콜릿도 주셔서 넘 행복한 생일 보낼 수 있었네요 ...,0
4,인터컨티넨탈 서울 코엑스,다 좋았는데 아침 식사는 갈수록 상태가 않좋아졌어요! 아침숙사는 그리 추천하지 않읍...,0
...,...,...,...
291,레드스톤 게스트 하우스,하하 뜻하지않게조카와친구를묶게했는데요 홍대주변에서놀다가들어갈수있는곳을물색하다가 첫째...,1
292,골드코스트호텔,"두번째 갔는데 좋았어요! 욕실도 깔끔하고 침구도 깔끔하고, 방음도 잘되었습니다 무엇...",1
293,라마다 서울 신도림 호텔,아주 마음에 드는 숙박이었어여^^ 다만 주말에는 웨딩홀땜에 혼잡해서 엘레베이터 타기...,1
294,킨스 하우스,간단한 취사도 좋고 깨끗함 넘 좋았네요 2박 수건도 여유롭게 아주만족합니다 다만 숙...,0


In [34]:
final_data['부정']=topic

In [35]:
merge_pos_neg=pd.merge(final_data2,final_data,on='호텔이름',how='inner')
del merge_pos_neg['리뷰_y']
merge_pos_neg.columns=['호텔이름','리뷰','긍정','부정']

In [36]:
merge_pos_neg['부정'].value_counts()

2    113
1     95
0     68
Name: 부정, dtype: int64

In [37]:
merge_pos_neg

,호텔이름,리뷰,긍정,부정
0,그랜드 머큐어 앰배서더 호텔 앤드 레지던스 서울 용산,만족합니다 서비스 청결 등등 모두 좋았습니다 단지 아쉬운점은 레지던스 호텔이다보니 ...,1,0
1,롯데 호텔 서울,로비에서부터 체크아웃 까지 정말 친절한 호텔 특별한 날에 이그제큐티브 룸 이용할 만...,1,1
2,남대문 호텔앤스위트,숙소가 상당히 깔끔하고 깨끗해서 좋았습니다 방 시설 좋았고 근처 인프라 좋아요 어매...,1,1
3,서울신라호텔,생일이라고 축하도 엄청해주시고 초콜릿도 주셔서 넘 행복한 생일 보낼 수 있었네요 ...,0,0
4,인터컨티넨탈 서울 코엑스,다 좋았는데 아침 식사는 갈수록 상태가 않좋아졌어요! 아침숙사는 그리 추천하지 않읍...,0,0
...,...,...,...,...
271,레드스톤 게스트 하우스,하하 뜻하지않게조카와친구를묶게했는데요 홍대주변에서놀다가들어갈수있는곳을물색하다가 첫째...,1,2
272,골드코스트호텔,"두번째 갔는데 좋았어요! 욕실도 깔끔하고 침구도 깔끔하고, 방음도 잘되었습니다 무엇...",1,0
273,라마다 서울 신도림 호텔,아주 마음에 드는 숙박이었어여^^ 다만 주말에는 웨딩홀땜에 혼잡해서 엘레베이터 타기...,1,0
274,킨스 하우스,간단한 취사도 좋고 깨끗함 넘 좋았네요 2박 수건도 여유롭게 아주만족합니다 다만 숙...,0,1


In [80]:
merge_pos_neg.to_excel('merge_pos_neg.xlsx',encoding='cp949',index=False)